In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
def get_fund_info(tickers):
    # Initialize lists to store data
    data = []
    
    for ticker in tickers:
        try:
            # Fetch fund info
            fund = yf.Ticker(ticker)
            info = fund.info

            # print(ticker)
            # for key in info.keys():
            #     print(key)
            
            # Get fund type and category
            fund_type = info.get('quoteType', 'N/A')
            fund_name = info.get('shortName', 'N/A')
            category = info.get('category', 'N/A')
            
            # Get expense ratio
            exp_ratio = info.get('annualReportExpenseRatio', 
                               info.get('totalExpenseRatio', np.nan))
            if exp_ratio:
                exp_ratio = exp_ratio * 100  # Convert to percentage
            
            # Get historical performance
            hist = fund.history(period="max")
            current_price = hist['Close'].iloc[-1]

            # Get inception date (first available date in historical data)
            inception_date = hist.index[0].strftime('%Y-%m-%d')
            
            # Calculate returns for different periods
            periods = {
                '1Y': {'days': 252, 'years': 1},    # Trading days in 1 year
                '3Y': {'days': 756, 'years': 3},    # Trading days in 3 years
                '5Y': {'days': 1260, 'years': 5},   # Trading days in 5 years
                '10Y': {'days': 2520, 'years': 10}, # Trading days in 10 years
            }
            
            returns = {}
            for period, info in periods.items():
                if len(hist) >= info['days']:
                    old_price = hist['Close'].iloc[-info['days']]
                    total_return = (current_price - old_price) / old_price
                    # Convert to annualized return
                    annualized_return = (((1 + total_return) ** (1/info['years'])) - 1) * 100
                    returns[period] = annualized_return
                else:
                    returns[period] = np.nan
            
            # Calculate since inception return (annualized)
            first_price = hist['Close'].iloc[0]
            total_return = (current_price - first_price) / first_price
            years_since_inception = (hist.index[-1] - hist.index[0]).days / 365.25
            since_inception = (((1 + total_return) ** (1/years_since_inception)) - 1) * 100
            
            # Compile data
            fund_data = {
                'Ticker': ticker,
                'Fund Name': fund_name,
                'Fund Type': fund_type,
                'Category': category,
                'Inception Date': inception_date,
                'Expense Ratio (%)': round(exp_ratio, 2) if not np.isnan(exp_ratio) else np.nan,
                '1Y Ann. Return (%)': round(returns.get('1Y', np.nan), 2),
                '3Y Ann. Return (%)': round(returns.get('3Y', np.nan), 2),
                '5Y Ann. Return (%)': round(returns.get('5Y', np.nan), 2),
                '10Y Ann. Return (%)': round(returns.get('10Y', np.nan), 2),
                'Since Inception Ann. Return (%)': round(since_inception, 2)
            }
            
            data.append(fund_data)
            
        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df


In [4]:
# Example usage
tickers = ['SPY', 'VTI', 'VFIAX','CLSE','EHLS','CTA','DBMF','HEQT','SVOL','EQLS']  # Replace with your fund tickers
fund_info_df = get_fund_info(tickers)
display(fund_info_df)

,Ticker,Fund Name,Fund Type,Category,Inception Date,Expense Ratio (%),1Y Ann. Return (%),3Y Ann. Return (%),5Y Ann. Return (%),10Y Ann. Return (%),Since Inception Ann. Return (%)
0,SPY,SPDR S&P 500,ETF,Large Blend,1993-01-29,NaN,25.59,8.65,14.51,12.84,10.45
1,VTI,Vanguard Total Stock Market ETF,ETF,Large Blend,2001-06-15,NaN,24.65,7.72,13.89,12.36,9.20
2,VFIAX,Vanguard 500 Index Fd Admiral S,MUTUALFUND,N/A,2000-11-13,0.04,25.28,8.58,14.46,12.86,8.23
3,CLSE,Trust for Professional Managers,ETF,Long-Short Equity,2022-02-22,NaN,36.87,NaN,NaN,NaN,16.44
4,EHLS,Even Herd Long Short ETF,ETF,Long-Short Equity,2024-04-02,NaN,NaN,NaN,NaN,NaN,15.78
5,CTA,Simplify Exchange Traded Funds,ETF,Systematic Trend,2022-03-08,NaN,22.54,NaN,NaN,NaN,10.64
6,DBMF,iMGP DBi Managed Futures Strate,ETF,Systematic Trend,2019-05-08,NaN,3.66,5.09,5.48,NaN,6.72
7,HEQT,Simplify Hedged Equity ETF,ETF,Equity Hedged,2021-11-02,NaN,18.25,8.05,NaN,NaN,8.46
8,SVOL,Simplify Volatility Premium ETF,ETF,Large Blend,2021-05-13,NaN,5.27,6.85,NaN,NaN,9.53
9,EQLS,Simplify Exchange Traded Funds,ETF,Equity Market Neutral,2023-06-14,NaN,-5.36,NaN,NaN,NaN,-5.42
